In [ ]:
import json
import psycopg2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# Database configuration
db_config = json.load(open('config.json'))

# Query to fetch data
query = """
SELECT 
    P.article_number,
    P.category,
    P.department,
    DC.d0_inventory,
    DC.d1_inventory,
    DC.d2_inventory,
    DC.d3_inventory,
    DC.d4_inventory,
    DC.d5_inventory,
    DC.d6_inventory
FROM 
    public.dailycheckin DC
JOIN 
    public.products P
ON 
    P.id = DC.product_id
ORDER BY 
    P.article_number, DC.year, DC.week;
"""

# FCNN Model
class FCNN(nn.Module):
    def __init__(self, num_products, num_categories, num_departments, embed_dim=8):
        super(FCNN, self).__init__()
        # Embedding layers
        self.product_embedding = nn.Embedding(num_products, embed_dim)
        self.category_embedding = nn.Embedding(num_categories, embed_dim)
        self.department_embedding = nn.Embedding(num_departments, embed_dim)
        
        # Fully connected layers
        self.fc1 = nn.Linear(embed_dim * 3 + 7, 64)  # Embeddings + 7 inventory features
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Output: Predict next day's inventory

    def forward(self, inventory_features, product_ids, category_ids, department_ids):
        # Embedding lookups
        product_emb = self.product_embedding(product_ids)
        category_emb = self.category_embedding(category_ids)
        department_emb = self.department_embedding(department_ids)
        
        # Combine embeddings and features
        combined = torch.cat((inventory_features, product_emb, category_emb, department_emb), dim=1)
        
        # Fully connected layers
        x = torch.relu(self.fc1(combined))
        x = torch.relu(self.fc2(x))
        output = self.fc3(x)
        return output

# Fetch and preprocess data
def fetch_data_from_db():
    conn = psycopg2.connect(**db_config)
    df = pd.read_sql(query, conn)
    conn.close()
    return df

def prepare_data(df):
    # Create mappings
    product_mapping = {prod: idx for idx, prod in enumerate(df['article_number'].unique())}
    category_mapping = {cat: idx for idx, cat in enumerate(df['category'].unique())}
    department_mapping = {dept: idx for idx, dept in enumerate(df['department'].unique())}

    # Map categorical data
    df['product_id'] = df['article_number'].map(product_mapping)
    df['category_id'] = df['category'].map(category_mapping)
    df['department_id'] = df['department'].map(department_mapping)

    # Prepare features and targets
    inventory_features = df[['d0_inventory', 'd1_inventory', 'd2_inventory', 
                              'd3_inventory', 'd4_inventory', 'd5_inventory', 
                              'd6_inventory']].fillna(0).values
    targets = df['d0_inventory'].shift(-1).fillna(0).values

    # Convert to tensors
    inventory_features_tensor = torch.tensor(inventory_features, dtype=torch.float)
    targets_tensor = torch.tensor(targets, dtype=torch.float)
    product_ids_tensor = torch.tensor(df['product_id'].values, dtype=torch.long)
    category_ids_tensor = torch.tensor(df['category_id'].values, dtype=torch.long)
    department_ids_tensor = torch.tensor(df['department_id'].values, dtype=torch.long)

    return inventory_features_tensor, targets_tensor, product_ids_tensor, category_ids_tensor, department_ids_tensor, \
           len(product_mapping), len(category_mapping), len(department_mapping)

# Train and Test
def train_and_test_model():
    print("Fetching data...")
    df = fetch_data_from_db()

    print("Preparing data...")
    inventory_features, targets, product_ids, category_ids, department_ids, num_products, num_categories, num_departments = prepare_data(df)

    # Train-test split
    X_train, X_test, y_train, y_test, prod_train, prod_test, cat_train, cat_test, dept_train, dept_test = train_test_split(
        inventory_features, targets, product_ids, category_ids, department_ids, test_size=0.2, random_state=42
    )

    # Initialize model, loss, optimizer
    model = FCNN(num_products, num_categories, num_departments)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    epochs = 10
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(X_train, prod_train, cat_train, dept_train).squeeze()
        loss = criterion(outputs, y_train)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

    # Testing loop
    model.eval()
    with torch.no_grad():
        test_predictions = model(X_test, prod_test, cat_test, dept_test).squeeze()
        test_loss = criterion(test_predictions, y_test)
        print(f"Test Loss: {test_loss.item()}")

if __name__ == "__main__":
    train_and_test_model()
